In [1]:

import os
import time
from PIL import Image, ImageOps
import numpy as np
import scipy.misc
from six.moves import urllib
import pandas as pd
import csv

def get_resized_image(img_path, width, height, save=True):
    image = Image.open(img_path)
    # PIL is column major so you have to swap the places of width and height
    image = ImageOps.fit(image, (width, height), Image.ANTIALIAS)
    if save:
        image_dirs = img_path.split('/')
        image_dirs[-1] = 'resized_' + image_dirs[-1]
        out_path = '/'.join(image_dirs)
        if not os.path.exists(out_path):
            image.save(out_path)
    image = np.asarray(image, np.float32)
    return np.expand_dims(image, 0)

def generate_noise_image(content_image, width, height, noise_ratio=0.6):
    noise_image = np.random.uniform(-20, 20, (1, height, width, 3)).astype(np.float32)
    return noise_image * noise_ratio + content_image * (1 - noise_ratio)

def save_image(path, image):
    image = image[0]
    image = np.clip(image, 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)

def safe_mkdir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass

In [2]:
import numpy as np
import scipy.io
import tensorflow as tf

class VGG(object):
    def __init__(self, input_img):
#         utils.download(VGG_DOWNLOAD_LINK, VGG_FILENAME, EXPECTED_BYTES)
        self.vgg_layers = scipy.io.loadmat('/home/n/projectdl/data/imagenet-vgg-verydeep-19.mat')['layers']
        self.input_img = input_img
        self.mean_pixels = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))


    def _weights(self, layer_idx, expected_layer_name):
        """ Return the weights and biases at layer_idx already trained by VGG
        """
        W = self.vgg_layers[0][layer_idx][0][0][2][0][0]
        b = self.vgg_layers[0][layer_idx][0][0][2][0][1]
        layer_name = self.vgg_layers[0][layer_idx][0][0][0][0]
        assert layer_name == expected_layer_name
        return W, b.reshape(b.size)

    def conv2d_relu(self, prev_layer, layer_idx, layer_name):
        """ Return the Conv2D layer with RELU using the weights, 
        biases from the VGG model at 'layer_idx'.
        Don't forget to apply relu to the output from the convolution.
        Inputs:
            prev_layer: the output tensor from the previous layer
            layer_idx: the index to current layer in vgg_layers
            layer_name: the string that is the name of the current layer.
                        It's used to specify variable_scope.


        Note that you first need to obtain W and b from from the corresponding VGG's layer 
        using the function _weights() defined above.
        W and b returned from _weights() are numpy arrays, so you have
        to convert them to TF tensors. One way to do it is with tf.constant.

        Hint for choosing strides size: 
            for small images, you probably don't want to skip any pixel
        """
        ###############################
        ## TO DO
        with tf.variable_scope(layer_name) as scope:
            W, b = self._weights(layer_idx, layer_name)
            W = tf.constant(W, name='weights')
            b = tf.constant(b, name='bias')
            conv2d = tf.nn.conv2d(prev_layer, 
                                filter=W, 
                                strides=[1, 1, 1, 1], 
                                padding='SAME')
            out = tf.nn.relu(conv2d + b)
        ###############################
        setattr(self, layer_name, out)

    def avgpool(self, prev_layer, layer_name):
        """ Return the average pooling layer. The paper suggests that 
        average pooling works better than max pooling.
        Input:
            prev_layer: the output tensor from the previous layer
            layer_name: the string that you want to name the layer.
                        It's used to specify variable_scope.

        Hint for choosing strides and kszie: choose what you feel appropriate
        """
        ###############################
        ## TO DO
        with tf.variable_scope(layer_name):
            out = tf.nn.avg_pool(prev_layer, 
                                ksize=[1, 2, 2, 1], 
                                strides=[1, 2, 2, 1],
                                padding='SAME')
        ###############################
        setattr(self, layer_name, out)

    def load(self):
        self.conv2d_relu(self.input_img, 0, 'conv1_1')
        self.conv2d_relu(self.conv1_1, 2, 'conv1_2')
        self.avgpool(self.conv1_2, 'avgpool1')
        self.conv2d_relu(self.avgpool1, 5, 'conv2_1')
        self.conv2d_relu(self.conv2_1, 7, 'conv2_2')
        self.avgpool(self.conv2_2, 'avgpool2')
        self.conv2d_relu(self.avgpool2, 10, 'conv3_1')
        self.conv2d_relu(self.conv3_1, 12, 'conv3_2')
        self.conv2d_relu(self.conv3_2, 14, 'conv3_3')
        self.conv2d_relu(self.conv3_3, 16, 'conv3_4')
        self.avgpool(self.conv3_4, 'avgpool3')
        self.conv2d_relu(self.avgpool3, 19, 'conv4_1')
        self.conv2d_relu(self.conv4_1, 21, 'conv4_2')
        self.conv2d_relu(self.conv4_2, 23, 'conv4_3')
        self.conv2d_relu(self.conv4_3, 25, 'conv4_4')
        self.avgpool(self.conv4_4, 'avgpool4')
        self.conv2d_relu(self.avgpool4, 28, 'conv5_1')
        self.conv2d_relu(self.conv5_1, 30, 'conv5_2')
        self.conv2d_relu(self.conv5_2, 32, 'conv5_3')
        self.conv2d_relu(self.conv5_3, 34, 'conv5_4')
        self.avgpool(self.conv5_4, 'avgpool5')


/home/n/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:

class StyleTransfer(object):
    def __init__(self, content_img, style_img, img_width, img_height,op):
       
        self.img_width = img_width
        self.img_height = img_height
        self.content_img = get_resized_image(content_img, img_width, img_height)
        self.content_img = self.content_img[:,:,:,0:3]
        self.style_img = get_resized_image(style_img, img_width, img_height)
        self.initial_img = generate_noise_image(self.content_img, img_width, img_height)

        ###############################
        ## TO DO
        ## create global step (gstep) and hyperparameters for the model
        self.content_layer = 'conv4_2'
        self.style_layers = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']
        self.content_w = 0.01
        self.style_w = 1
        self.style_layer_w = [0.5, 1.0, 1.5, 3.0, 4.0] 
        self.gstep = tf.Variable(0, dtype=tf.int32, 
                                trainable=False, name='global_step')
        
        ###############################
        self.opp = 'lbfgs'
        
        
        
        
        

    def create_input(self):
        '''
        We will use one input_img as a placeholder for the content image, 
        style image, and generated image, because:
            1. they have the same dimension
            2. we have to extract the same set of features from them
        We use a variable instead of a placeholder because we're, at the same time, 
        training the generated image to get the desirable result.

        Note: image height corresponds to number of rows, not columns.
        '''
        with tf.variable_scope('input') as scope:
            self.input_img = tf.get_variable('in_img', 
                                        shape=([1, self.img_height, self.img_width, 3]),
                                        dtype=tf.float32,
                                        initializer=tf.zeros_initializer())
            
    def load_vgg(self):
        '''
        Load the saved model parameters of VGG-19, using the input_img
        as the input to compute the output at each layer of vgg.

        During training, VGG-19 mean-centered all images and found the mean pixels
        to be [123.68, 116.779, 103.939] along RGB dimensions. We have to subtract
        this mean from our images.


        On passing any image from content, style, initial image it will create the model for that image
        '''
        self.vgg = VGG(self.input_img)
        self.vgg.load()
        self.content_img -= self.vgg.mean_pixels
        self.style_img -= self.vgg.mean_pixels
        
        
    def _content_loss(self, P):
        ''' Calculate the loss between the feature representation of the
        content image and the generated image.
        
        Inputs: 
            P: content representation of the content image
            F: content representation of the generated image
            Read the assignment handout for more details

            Note: Don't use the coefficient 0.5 as defined in the paper.
            Use the coefficient defined in the assignment handout.
        '''
        # self.content_loss = None
        ###############################
        ## TO DO
        F = getattr(self.vgg,self.content_layer)
        self.content_loss = tf.reduce_sum((F - P) ** 2) / (4.0)

        
    def _gram_matrix(self, F, N, M):
        """ Create and return the gram matrix for tensor F
            Hint: you'll first have to reshape F
        """
        ###############################
        ## TO DO
        F = tf.reshape(F, (M, N))
        return tf.matmul(tf.transpose(F), F)
        ###############################

        
    def _single_style_loss(self, a, g):
        """ Calculate the style loss at a certain layer
        Inputs:
            a is the feature representation of the style image at that layer
            g is the feature representation of the generated image at that layer
        Output:
            the style loss at a certain layer (which is E_l in the paper)

        Hint: 1. you'll have to use the function _gram_matrix()
            2. we'll use the same coefficient for style loss as in the paper
            3. a and g are feature representation, not gram matrices
        """
        ###############################
        ## TO DO
        N = a.shape[3] # number of filters
        M = a.shape[1] * a.shape[2] # height times width of the feature map
        A = self._gram_matrix(a, N, M)
        G = self._gram_matrix(g, N, M)
        return tf.reduce_sum((G - A) ** 2 / ((2 * N * M) ** 2))

    
    def _style_loss(self, A):
        """ Calculate the total style loss as a weighted sum 
        of style losses at all style layers
        Hint: you'll have to use _single_style_loss()
        """
        n_layers = len(A)
        E = [self._single_style_loss(A[i], getattr(self.vgg, self.style_layers[i])) for i in range(n_layers)]
        
        ###############################
        ## TO DO
        self.style_loss = sum([self.style_layer_w[i] * E[i] for i in range(n_layers)])

    def losses(self):
        with tf.variable_scope('losses') as scope:
            with tf.Session() as sess:
                # assign content image to the input variable
                sess.run(self.input_img.assign(self.content_img)) 
                gen_img_content = getattr(self.vgg, self.content_layer)
#                 content_img_content = sess.run(gen_img_content)
            self._content_loss(gen_img_content)

            with tf.Session() as sess:
                sess.run(self.input_img.assign(self.style_img))
                style_layers = sess.run([getattr(self.vgg, layer) for layer in self.style_layers])                              
            self._style_loss(style_layers)

            ##########################################
            ## TO DO: create total loss. 
            ## Hint: don't forget the weights for the content loss and style loss
            self.total_loss = self.content_w * self.content_loss + self.style_w * self.style_loss
            ##########################################
    def optimize(self,op):
        ###############################
        ## TO DO: create optimizer
        if op==1:
            self.lr = 2.0
            self.opt = tf.train.AdamOptimizer(self.lr).minimize(self.total_loss,global_step=self.gstep)
            
        elif op==2:
            self.lr = 2.0
            self.opt = tf.train.AdagradOptimizer(self.lr).minimize(self.total_loss,global_step=self.gstep)
        elif op==3:
            self.lr = 2.0
            self.opt = tf.train.AdadeltaOptimizer(self.lr).minimize(self.total_loss,global_step=self.gstep)
        elif op == 4:
            self.lr = 2.0
            self.opt = tf.train.RMSPropOptimizer(self.lr).minimize(self.total_loss,global_step=self.gstep)
        elif op == 5:
            self.lr = 0.1
            self.opt = tf.train.MomentumOptimizer(self.lr,0.9).minimize(self.total_loss,global_step=self.gstep)
        elif op == 6:
            self.opt = tf.contrib.opt.ScipyOptimizerInterface(
            self.total_loss, method='L-BFGS-B',
            options={'maxiter': 0})
            self.opt
            self.opp = "2nd"

        ###############################
#     def minimize_with_lbfgs(sess):
# #     if args.verbose: print('\nMINIMIZING LOSS USING: L-BFGS OPTIMIZER')
# #     init_op = tf.global_variables_initializer()
# #     sess.run(init_op)
#         sess.run(net['input'].assign(init_img))
#         optimizer.minimize(sess)


#     def get_optimizer(self):
#     #     print_iterations = args.print_iterations if args.verbose else 0
#         if self.opp == 'lbfgs':
#             self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(
#             self.total_loss, method='L-BFGS-B',
#             options={'maxiter': 1})
#             self.optimizer
    def create_summary(self):
        ###############################
        ## TO DO: create summaries for all the losses
        ## Hint: don't forget to merge them
        with tf.name_scope('summaries'):
            tf.summary.scalar('content loss', self.content_loss)
            tf.summary.scalar('style loss', self.style_loss)
            tf.summary.scalar('total loss', self.total_loss)
            self.summary_op = tf.summary.merge_all()
        ###############################


    def build(self,op):
        self.create_input()
        self.load_vgg()
        self.losses()
        self.optimize(op)
#         self.get_optimizer()
        self.create_summary()

    def train(self, n_iters):
        skip_step = 1
        with tf.Session() as sess:
            
            ###############################
            ## TO DO: 
            ## 1. initialize your variables
            ## 2. create writer to write your graph
            sess.run(tf.global_variables_initializer())
            writer = tf.summary.FileWriter('graphs/style_stranfer', sess.graph)
            ###############################
            sess.run(self.input_img.assign(self.initial_img))


            ###############################
            ## TO DO: 
            ## 1. create a saver object
            ## 2. check if a checkpoint exists, restore the variables
            saver = tf.train.Saver()
            ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/style_transfer/checkpoint'))
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
            ##############################

            initial_step = self.gstep.eval()
            
            start_time = time.time()
            for index in range(initial_step, n_iters):
                if index >= 5 and index < 20:
                    skip_step = 10
                elif index >= 20:
                    skip_step = 20
                if(self.opp == "2nd"):
                    self.opt.minimize(sess)
                else:
                    sess.run(self.opt)
                
                
                if (index + 1) % skip_step == 0:
                    ###############################
                    ## TO DO: obtain generated image, loss, and summary
                    gen_image, total_loss, summary = sess.run([self.input_img,
                                                                self.total_loss,
                                                                self.summary_op])
                    print(self.style_loss.eval(session=sess))


                    ###############################
                    
                    # add back the mean pixels we subtracted before
                    gen_image = gen_image + self.vgg.mean_pixels 
                    writer.add_summary(summary, global_step=index)
                    print('Step {}\n   Sum: {:5.1f}'.format(index + 1, np.sum(gen_image)))
                    print('   Loss: {:5.1f}'.format(total_loss))
                    print('   Took: {} seconds'.format(time.time() - start_time))
                    start_time = time.time()

                    filename = 'outputs/%d.png' % (index)
                    save_image(filename, gen_image)
                    myData = [[index,total_loss/np.sum(gen_image)]  ]
                    myFile = open('/home/n/projectdl/19adam.csv', 'a')  
                    with myFile:  
                        writer_ = csv.writer(myFile)
                        writer_.writerows(myData)
                    

                    if (index + 1) % 20 == 0:
                        ###############################
                        ## TO DO: save the variables into a checkpoint
                        saver.save(sess, 'checkpoints/style_stranfer/style_transfer', index)
                        ###############################

if __name__ == '__main__':
#     setup()
    op = int(input("enter optimizer"))
#     op = 1
    machine = StyleTransfer('/home/n/projectdl/content1.jpg', '/home/n/projectdl/color4.jpg',500,500,op)
    machine.build(op)
    machine.train(500)

    



enter optimizer1
INFO:tensorflow:Summary name content loss is illegal; using content_loss instead.
INFO:tensorflow:Summary name style loss is illegal; using style_loss instead.
INFO:tensorflow:Summary name total loss is illegal; using total_loss instead.
10378508000.0
Step 1
   Sum: 120366222.5
   Loss: 10378508288.0
   Took: 8.868210077285767 seconds


/home/n/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


9469638000.0
Step 2
   Sum: 120457418.2
   Loss: 9469637632.0
   Took: 0.575810432434082 seconds
8584993000.0
Step 3
   Sum: 120549427.5
   Loss: 8584992768.0
   Took: 0.5433416366577148 seconds
7741278700.0
Step 4
   Sum: 120642545.1
   Loss: 7741278720.0
   Took: 0.5419623851776123 seconds
6956759000.0
Step 5
   Sum: 120736863.1
   Loss: 6956759040.0
   Took: 0.5413942337036133 seconds
4083765800.0
Step 10
   Sum: 121229827.1
   Loss: 4083765760.0
   Took: 1.5297889709472656 seconds
2118965400.0
Step 20
   Sum: 122072982.5
   Loss: 2118965376.0
   Took: 2.75248646736145 seconds
928066700.0
Step 40
   Sum: 122409709.0
   Loss: 928066688.0
   Took: 5.635805130004883 seconds
568697000.0
Step 60
   Sum: 122116030.3
   Loss: 568697024.0
   Took: 6.1074113845825195 seconds
402347170.0
Step 80
   Sum: 121676936.1
   Loss: 402347168.0
   Took: 6.040081739425659 seconds
306891520.0
Step 100
   Sum: 121266572.3
   Loss: 306891520.0
   Took: 6.1307854652404785 seconds
245012130.0
Step 120
   Su

### https://github.com/machrisaa/tensorflow-vgg

In [4]:
minimize_with_lbfgs(sess, net, optimizer, init_img)
def minimize_with_lbfgs(sess, net, optimizer, init_img):
    if args.verbose: print('\nMINIMIZING LOSS USING: L-BFGS OPTIMIZER')
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    sess.run(net['input'].assign(init_img))
    optimizer.minimize(sess)


def get_optimizer(loss):
    print_iterations = args.print_iterations if args.verbose else 0
    if args.optimizer == 'lbfgs':
        optimizer = tf.contrib.opt.ScipyOptimizerInterface(
        loss, method='L-BFGS-B',
        options={'maxiter': args.max_iterations,
                  'disp': print_iterations})
    elif args.optimizer == 'adam':
        optimizer = tf.train.AdamOptimizer(args.learning_rate)
    return optimizer

 output_img = sess.run(net['input'])

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 21)

In [ ]:
minimize_with_lbfgs(sess, net, optimizer, init_img)
def minimize_with_lbfgs(sess):
#     if args.verbose: print('\nMINIMIZING LOSS USING: L-BFGS OPTIMIZER')
#     init_op = tf.global_variables_initializer()
#     sess.run(init_op)
    sess.run(net['input'].assign(init_img))
    optimizer.minimize(sess)


def get_optimizer(self):
#     print_iterations = args.print_iterations if args.verbose else 0
    if self.opp == 'lbfgs':
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(
        self.total_loss, method='L-BFGS-B',
        options={'maxiter': 10})
        self.optimizer

output_img = sess.run(net['input'])